# Proyecto análisis de conversación de mensajería

## Cargar datos
- Exportar una conversación personal o grupal de Whastapp o Telegram en un formato adecuado (JSON, CSV, etc), no exportar audios, archivos multimedia, etc
- Importar archivo y convertirlo en dataframe
- Obtener y graficar los siguientes datos. Elegir el gráfico que mejor represente los datos

In [1]:
import json
import pandas as pd
import numpy as np
import requests
import os, sys
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import dateutil


bd2 = open('../data/result.json', encoding='utf8')
data = json.load(bd2)


msgs = data['messages']
df = pd.DataFrame(msgs)

df = df.filter(items = ['id','date', 'from', 'reply_to_message_id' ,'text'])
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
df.sample(5)

,id,date,from,reply_to_message_id,text
2580,2594,2020-05-27 12:39:48,Diego Córdoba (at @juncotic),NaN,"yep, eso debería sacarlo para este ejemplo"
3393,3412,2021-08-11 16:48:32,Diego Córdoba (at @juncotic),NaN,[Gente! Ya tienen publicados los videos de la ...
3192,3210,2021-03-17 12:00:37,jose Ruti,NaN,Okey igual apenas me vuelva entro de vuelta
3261,3279,2021-04-07 11:48:51,Diego Córdoba (at @juncotic),NaN,
2635,2649,2020-05-27 12:55:57,Ornella Grasso,NaN,deberia ver si tengo el archivito donde tenia ...


### Cantidad de mensajes enviados por cada usuario

In [2]:
df_msg = df.groupby('from')['text'].count()
df_msg

from
Axel Guyot                         5
Bruno                              1
Bruno Rosales                     15
Daiana Ascurra                    26
Danilo Verardo                     8
Delfina Quinteros                 11
Diego Córdoba (at @juncotic)    1593
Douglas Arenas                    15
Emiliano Muñoz                     1
Esteban                          292
Fede Pardo                        11
Giancarlo Galvarini              148
Harry                              3
Juan I. Castellano                95
Juan Pablo Garcia                 17
Julián                            33
Lauti Giménez                      2
Lucas Galdame Villegas            11
Lucas Ollarce                      4
Lucila Martínez                   20
Luis A. Airabella                 20
Marcos Adrián Rugoso              16
Mati Ahumada                       1
Matias Ruiz                      108
Matías Romani                     47
Nico Olivares                     11
Ornella Grasso                   

### Promedio de palabras utilizadas por mensaje por cada usuario

In [3]:
df_text = df['text']
quantity = 0
content = ''
for item in df_text:
    if not item == '':
        if type(item) == list:
            if type(item[0]) == dict:
                content = item[0]['text']
                quantity += 1
        else:
            content = item.split(' ')
            quantity += len(content)
        
print(f'Cantidad de palabras: {quantity}')
total_msg = df_msg.sum()
print(f'Cantidad de mensajes: {total_msg}')

total = quantity/total_msg
print(f'Se ulilizaron un promedio de: {total} palabras por mensaje.')

df.groupby('from')['text'].count() * total

Cantidad de palabras: 25992
Cantidad de mensajes: 3185
Se ulilizaron un promedio de: 8.160753532182104 palabras por mensaje.


from
Axel Guyot                         40.803768
Bruno                               8.160754
Bruno Rosales                     122.411303
Daiana Ascurra                    212.179592
Danilo Verardo                     65.286028
Delfina Quinteros                  89.768289
Diego Córdoba (at @juncotic)    13000.080377
Douglas Arenas                    122.411303
Emiliano Muñoz                      8.160754
Esteban                          2382.940031
Fede Pardo                         89.768289
Giancarlo Galvarini              1207.791523
Harry                              24.482261
Juan I. Castellano                775.271586
Juan Pablo Garcia                 138.732810
Julián                            269.304867
Lauti Giménez                      16.321507
Lucas Galdame Villegas             89.768289
Lucas Ollarce                      32.643014
Lucila Martínez                   163.215071
Luis A. Airabella                 163.215071
Marcos Adrián Rugoso              130.572057
Mati 

### Porcentaje de mensajes enviados por cada usuario

In [4]:
percentage = (df.groupby('from')['text'].count() / total_msg) * 100
percentage

from
Axel Guyot                       0.156986
Bruno                            0.031397
Bruno Rosales                    0.470958
Daiana Ascurra                   0.816327
Danilo Verardo                   0.251177
Delfina Quinteros                0.345369
Diego Córdoba (at @juncotic)    50.015699
Douglas Arenas                   0.470958
Emiliano Muñoz                   0.031397
Esteban                          9.167975
Fede Pardo                       0.345369
Giancarlo Galvarini              4.646782
Harry                            0.094192
Juan I. Castellano               2.982732
Juan Pablo Garcia                0.533752
Julián                           1.036107
Lauti Giménez                    0.062794
Lucas Galdame Villegas           0.345369
Lucas Ollarce                    0.125589
Lucila Martínez                  0.627943
Luis A. Airabella                0.627943
Marcos Adrián Rugoso             0.502355
Mati Ahumada                     0.031397
Matias Ruiz                  

### Frecuencia de envío de mensaje dependiendo la hora del día

In [6]:
groups = [df['date'].dt.to_period('h')]
agregations = {'text':["count"]}
df.groupby(groups).agg(agregations)

,text
,count
date,
2016-08-18 12:00,3
2016-08-18 13:00,6
2016-08-18 18:00,12
2016-08-18 21:00,1
2016-08-18 22:00,2
...,...
2021-09-01 10:00,1
2021-09-03 18:00,9


### Frecuencia de envío de mensaje dependiendo del día de la semana

In [10]:
weeks = {
0 : 'Lunes',
1 : 'Martes',
2 : 'Miercoles',
3 : 'Jueves',
4 : 'Viernes',
5 : 'Sabado',
6 : 'Domingo'
}
day = [df['date'].dt.weekday.map(weeks)]
agregations = {'text':"count"}
df.groupby(day).agg(agregations)

,text
date,
Domingo,18
Jueves,176
Lunes,148
Martes,223
Miercoles,2765
Sabado,22
Viernes,99


La mayoria de los mensajes fueron enviados el dia Miercoles.

### Hora del día preferida por cada usuario para enviar mensajes

In [8]:
groups = [df['date'].dt.to_period('h'),"from"]
agregations = {'text':'count'}
df.groupby(groups).agg(agregations)

text
date             from                              
2016-08-18 12:00 Diego Córdoba (at @juncotic)     1
2016-08-18 18:00 Diego Córdoba (at @juncotic)     6
2016-08-18 21:00 Ramiro Alberto Giandinoto        1
2016-08-18 22:00 Diego Córdoba (at @juncotic)     2
2016-08-18 23:00 Ramiro Alberto Giandinoto        1
...                                             ...
2021-09-08 09:00 Diego Córdoba (at @juncotic)     1
2021-09-08 17:00 Diego Córdoba (at @juncotic)     2
                 Julián                           1
                 Tato Beato                       1
                 Tomas Gañan                      1

[779 rows x 1 columns]

### Día de la semana preferida por cada usuario para enviar mensajes

In [11]:
groups = [df['date'].dt.to_period('w'),"from"]
agregations = {'text': "count"}
df.groupby(groups).agg(agregations)

text
date                  from                              
2016-08-15/2016-08-21 Diego Córdoba (at @juncotic)    11
                      Ramiro Alberto Giandinoto        2
2016-08-22/2016-08-28 Diego Córdoba (at @juncotic)     8
                      Ramiro Alberto Giandinoto        1
2016-08-29/2016-09-04 Diego Córdoba (at @juncotic)     6
...                                                  ...
2021-08-30/2021-09-05 Tato Beato                       2
2021-09-06/2021-09-12 Diego Córdoba (at @juncotic)     3
                      Julián                           1
                      Tato Beato                       1
                      Tomas Gañan                      1

[372 rows x 1 columns]

### Cantidad promedio de mensajes enviados por hora para cada usuario

In [12]:
groups = [df['date'].dt.to_period('h'),"from"]
agregations = {'text': ["count"]}
df.groupby(groups).agg(agregations)

,,text
,,count
date,from,
2016-08-18 12:00,Diego Córdoba (at @juncotic),1
2016-08-18 18:00,Diego Córdoba (at @juncotic),6
2016-08-18 21:00,Ramiro Alberto Giandinoto,1
2016-08-18 22:00,Diego Córdoba (at @juncotic),2
2016-08-18 23:00,Ramiro Alberto Giandinoto,1
...,...,...
2021-09-08 09:00,Diego Córdoba (at @juncotic),1
2021-09-08 17:00,Diego Córdoba (at @juncotic),2


### Cantidad de usuarios promedio que envían mensaje por cada día de la semana

In [14]:
groups = [df['date'].dt.weekday.map(weeks), 'from']
agregations = {'text':["count"]}
df.groupby(groups).agg(agregations)

text
                                     count
date    from                              
Domingo Daiana Ascurra                   1
        Diego Córdoba (at @juncotic)     6
        Esteban                          2
        Juan Pablo Garcia                1
        Matias Ruiz                      2
...                                    ...
Viernes Ornella Grasso                   5
        Pablo Fracaro                    2
        Ramiro Alberto Giandinoto        2
        Seba Fernández                   1
        Tato Beato                       2

[123 rows x 1 columns]

### Cantidad de mensajes envíados por mes

In [15]:
groups = [df['date'].dt.to_period('m'),"from"]
agregations = {'text': "count"}
df.groupby(groups).agg(agregations)

text
date    from                              
2016-08 Diego Córdoba (at @juncotic)    25
        Ramiro Alberto Giandinoto        3
2016-09 Diego Córdoba (at @juncotic)    16
        Ramiro Alberto Giandinoto        1
2016-10 Diego Córdoba (at @juncotic)    20
...                                    ...
2021-09 Douglas Arenas                   3
        Julián                           1
        Marcos Adrián Rugoso             2
        Tato Beato                       3
        Tomas Gañan                      1

[201 rows x 1 columns]

### Cantidad mínima, máxima y promedio de palabras por mensaje

In [21]:
df_text = df['text']
content = ''
max_size_text = []
words_max = 0
quantity_total = 0
for item in df_text:
    quantity = 0
    if not item == '':
        if type(item) == list:
            if type(item[0]) == dict:
                content = item[0]['text']
                quantity += 1
                quantity_total += quantity
                # print(f'{content} | Palabras: {quantity}')
        else:
            content = item.split(' ')
            quantity += len(content)
            quantity_total += quantity
            # print(f'{content} | Palabras: {quantity}')
            words_max_aux = len(content)
            if words_max < words_max_aux:
                words_max = words_max_aux
                max_size_text = content

print(f'Texto con el (Maximo) de palabras contiene: {words_max} palabras.')
total = quantity_total/df_msg.sum()
print(f'Se ulilizaron un promedio de: {total} palabras por mensaje.')

Texto con el (Maximo) de palabras contiene: 126 palabras.
Se ulilizaron un promedio de: 8.160753532182104 palabras por mensaje.
